# `MEDUSA`

### Setup and definitions

First, we set up a database with temporally distributed exchanges.

In [7]:
#I need to set working directory on my computer for from tests.databases import * to work
import os

current_directory = os.getcwd()
print("Current working directory:", current_directory)
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)
print("New working directory:", os.getcwd())

Current working directory: c:\Users\muell\Documents\PhD\Futuralis
New working directory: c:\Users\muell\Documents\PhD


First, no biosphere flow in the background

In [2]:

from tests.databases import *
db_abcd_CO2_and_CH4_foreground()

from datetime import datetime
database_date_dict = {
    "background_2008": datetime.strptime("2008", "%Y"),
    "background_2024": datetime.strptime("2024", "%Y"),
    "foreground": "dynamic",  # flag databases that should be temporally distributed with "dynamic"
}

SKIPPABLE = [
    node.id for node in bd.Database('background_2008')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

demand = {('foreground', 'A'): 1}
method = ('GWP', 'example')

from medusa import MedusaLCA
flca = MedusaLCA(demand, method, filter_function, database_date_dict, max_calc=5000)

import warnings
warnings.filterwarnings("ignore")
flca.build_timeline()

flca.build_datapackage()

flca.lci()
flca.lcia()

print('Old static LCA Score:', flca.static_lca.score)
print('New MEDUSA LCA Score:', flca.score)

100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<00:00, 2235.77it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Starting graph traversal
Calculation count: 5
All databases in database_date_dict exist as brightway project databases
Old static LCA Score: 54.30000203847885
New MEDUSA LCA Score: 54.30000203847885


score is the same between static and medua

now, with biosphere flows in the background, which are the same in both databases and have no additional biosphere TD;

In [3]:

from tests.databases import *
db_abcd_CO2_foreground_and_background()

from datetime import datetime
database_date_dict = {
    "background_2008": datetime.strptime("2008", "%Y"),
    "background_2024": datetime.strptime("2024", "%Y"),
    "foreground": "dynamic",  # flag databases that should be temporally distributed with "dynamic"
}

SKIPPABLE = [
    node.id for node in bd.Database('background_2008')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

demand = {('foreground', 'A'): 1}
method = ('GWP', 'example')

from medusa import MedusaLCA
mlca = MedusaLCA(demand, method, filter_function, database_date_dict, max_calc=5000)

import warnings
warnings.filterwarnings("ignore")
mlca.build_timeline()



100%|██████████| 2/2 [00:00<00:00, 1807.89it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<00:00, 5533.38it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


...

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 2005.40it/s]

Vacuuming database 


Starting graph traversal
Calculation count: 5
All databases in database_date_dict exist as brightway project databases


,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2018,10,2018-01-01,4,D,2024,12,2024-01-01,9,B,6.0,"{'background_2008': 0.37491444216290215, 'back..."
1,2023,11,2023-01-01,3,C,2024,12,2024-01-01,9,B,1.0,"{'background_2008': 0.06245722108145102, 'back..."
2,2024,12,2024-01-01,9,B,2024,13,2024-01-01,10,A,1.0,{'background_2024': 1}
3,2024,13,2024-01-01,10,A,2024,-1,2024-01-01,-1,-1,1.0,{'background_2024': 1}


In [6]:
mlca.build_datapackage()

mlca.lci()
mlca.lcia()

print('Old static LCA Score:', mlca.static_lca.score)
print('New MEDUSA LCA Score:', mlca.score)

Old static LCA Score: 33.5
New MEDUSA LCA Score: 239.49999999999997


Score should be different than in the calculation before, but there should be no difference between the static LCA score and the MEDUSA LCA score

In [4]:
mlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2018,10,2018-01-01,4,D,2024,12,2024-01-01,9,B,6.0,"{'background_2008': 0.37491444216290215, 'back..."
1,2023,11,2023-01-01,3,C,2024,12,2024-01-01,9,B,1.0,"{'background_2008': 0.06245722108145102, 'back..."
2,2024,12,2024-01-01,9,B,2024,13,2024-01-01,10,A,1.0,{'background_2024': 1}
3,2024,13,2024-01-01,10,A,2024,-1,2024-01-01,-1,-1,1.0,{'background_2024': 1}


In [5]:
mlca.create_labelled_technosphere_dataframe()

,"((background_2024, D), 2024)","((background_2024, electricity_wind), 2024)","((background_2008, C), 2008)","((background_2008, D), 2008)","((background_2008, electricity_wind), 2008)","((foreground, B), dynamic)","((foreground, A), dynamic)","((background_2024, D), 2018)","((background_2024, C), 2023)","((foreground, B), 2024)","((foreground, A), 2024)"
"((background_2024, D), 2024)",1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.000000,-0.937543,0.0,0.0
"((background_2024, electricity_wind), 2024)",0.0,1.0,0.0,0.0,0.0,0.0,-6.0,-0.625086,0.000000,0.0,0.0
"((background_2008, C), 2008)",-1.0,-3.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"((background_2008, D), 2008)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,-0.062457,0.0,0.0
"((background_2008, electricity_wind), 2008)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.374914,0.000000,0.0,0.0
"((foreground, B), dynamic)",0.0,0.0,0.0,-1.0,-3.0,1.0,0.0,0.000000,0.000000,0.0,0.0
"((foreground, A), dynamic)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.000000,0.000000,0.0,0.0
"((background_2024, D), 2018)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,-6.0,0.0
"((background_2024, C), 2023)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,-1.0,0.0
"((foreground, B), 2024)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,-1.0


In the above technosphere matrix, the temporal market for D for 2018 (column 8) has receives -1 from (foreground A, dynamic), which is wrong. There is no connection between the processes. This does not happen to a second temporal market at process C. I do not know where this comes from.

In [6]:
mlca.create_labelled_biosphere_dataframe()


,"((background_2024, electricity_wind), 2024)","((background_2008, C), 2008)","((background_2008, electricity_wind), 2008)","((foreground, B), dynamic)","((foreground, A), dynamic)","((background_2024, C), 2023)","((foreground, B), 2024)","((foreground, A), 2024)"
1,5.0,1.5,5.0,1.5,0.0,5.0,0.0,0.0
